In [9]:
from PIL import Image
import math
from torch.autograd import Variable
import os
import torch
from torch.autograd import Variable
import numpy as np
from options import vaeSROption
import utils
import filters
from train_block_vaeSRnet import blindVAE
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import matplotlib
# pretrain model

modeldir = "/home/wcd/Desktop/to_neural_style/ckpt/block/nonactive/come18_st2960000_block_vaeSR_gsig_3.0_glen_3_mang_89_mlen8_SRIN_block4_PBN_block8.model"
# setting & load net
torch.cuda.set_device(0)
opt = vaeSROption()
opt.batch_size = 1
opt.pretrain_model = modeldir
net = blindVAE(opt,isTrain=False,isSeperate=False,fromPretrain=False)
net.load_network(modeldir)
netP = net.netP
netSR = net.netSR
# img 
lrimg = Image.open("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/comic.bmp").convert('L')
hrimg = utils.load_HR_image("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/comic.bmp",mode='L')
lrtensor = transform(lrimg).unsqueeze(0)
lrtensor = (lrtensor - 0.5) / 0.5
lrvar = nn.Parameter(lrtensor.cuda())

# gd method
mseloss = nn.MSELoss(size_average=True).cuda()
optimizer = optim.Adam([lrvar], lr = 0.001)


------------ Networks initialized -------------
condition_SRNet (
  (resblocks): ModuleList (
    (0): ResidualBlock (
      (conv1): ConvLayer (
        (reflection_pad): ReflectionPad2d (1, 1, 1, 1)
        (conv2d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      )
      (conv2): ConvLayer (
        (reflection_pad): ReflectionPad2d (1, 1, 1, 1)
        (conv2d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      )
      (relu): ReLU ()
      (norm1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False)
      (norm2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False)
    )
    (1): ResidualBlock (
      (conv1): ConvLayer (
        (reflection_pad): ReflectionPad2d (1, 1, 1, 1)
        (conv2d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      )
      (conv2): ConvLayer (
        (reflection_pad): ReflectionPad2d (1, 1, 1, 1)
        (conv2d): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      )
      (relu): ReLU ()
      (norm1): InstanceNorm2

In [13]:
# derectly update lr
# img 
lrimg = Image.open("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/starfish.tif").convert('L')
hrimg = utils.load_HR_image("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/starfish.tif",mode='L')
lrtensor = transform(lrimg).unsqueeze(0)
lrtensor = (lrtensor - 0.5) / 0.5
lrvar = nn.Parameter(lrtensor.cuda())

# gd method
mseloss = nn.MSELoss(size_average=True).cuda()
optimizer = optim.Adam([lrvar], lr = 0.001)

lrimg = lrvar.data[0,0,:,:].clone().clamp(-1, 1).cpu()
lrimg = (lrimg + 1) / 2.0 * 255.0
lrimg = lrimg.numpy().astype("uint8")
lrimg = Image.fromarray(lrimg, 'L')
lrimg.show()
for run in range(10000):

    # correct the values of updated input image
    updated = lrvar.data.cpu().clone()
    updated = updated.numpy()
    updated[updated<-1] = -1
    updated[updated>1] = 1
    lrvar.data = torch.from_numpy(updated).cuda()

    optimizer.zero_grad()
    fakep = netP(lrvar)
    zerovar = Variable(torch.zeros(fakep.size()).cuda())
    ploss = mseloss(fakep,zerovar)
    ploss.backward()

    optimizer.step()

    if run%500==0:
        lrimg = lrvar.data[0,0,:,:].clone().clamp(-1, 1).cpu()
        lrimg = (lrimg + 1) / 2.0 * 255.0
        lrimg = lrimg.numpy().astype("uint8")
        lrimg = Image.fromarray(lrimg, 'L')
        lrimg.show()
        print(ploss)

Variable containing:
 0.2399
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-03 *
  1.1070
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  7.5987
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  6.0836
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  5.1597
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  4.5625
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  4.1485
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.8275
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.5612
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.3214
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.1027
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  2.9008
[to

In [16]:
# if ptensor is useful
lrimg = Image.open("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/starfish.tif").convert('L')
hrimg = utils.load_HR_image("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/starfish.tif",mode='L')
lrtensor = transform(lrimg).unsqueeze(0)
lrtensor = (lrtensor - 0.5) / 0.5
lrvar = Variable(lrtensor.cuda())
fakep = netP(lrvar)
zerovar = Variable(torch.zeros(fakep.size()).cuda())
srvar = netSR(lrvar,fakep)
zerosrvar = netSR(lrvar,zerovar)


lrimg = srvar.data[0,0,:,:].clone().clamp(-1, 1).cpu()
lrimg = (lrimg + 1) / 2.0 * 255.0
lrimg = lrimg.numpy().astype("uint8")
lrimg = Image.fromarray(lrimg, 'L')
lrimg.show()
        
lrimg = zerosrvar.data[0,0,:,:].clone().clamp(-1, 1).cpu()
lrimg = (lrimg + 1) / 2.0 * 255.0
lrimg = lrimg.numpy().astype("uint8")
lrimg = Image.fromarray(lrimg, 'L')
lrimg.show()

In [12]:
# derectly update sr
lrimg = Image.open("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/starfish.tif").convert('L')
hrimg = utils.load_HR_image("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/starfish.tif",mode='L')
lrtensor = transform(lrimg).unsqueeze(0)
lrtensor = (lrtensor - 0.5) / 0.5
lrvar = Variable(lrtensor.cuda())
fakep = netP(lrvar)
srvar = netSR(lrvar,fakep)
srlrvar = nn.Parameter(srvar.detach().data)
lrimg = srlrvar.data[0,0,:,:].clone().clamp(-1, 1).cpu()
lrimg = (lrimg + 1) / 2.0 * 255.0
lrimg = lrimg.numpy().astype("uint8")
lrimg = Image.fromarray(lrimg, 'L')
lrimg.show()
        
# gd method
mseloss = nn.MSELoss(size_average=True).cuda()
optimizer = optim.Adam([srlrvar], lr = 0.001)
for run in range(10000):
    # correct the values of updated input image
    optimizer.zero_grad()

    updated = srlrvar.data.cpu().clone()
    updated = updated.numpy()
    updated[updated<-1] = -1
    updated[updated>1] = 1
    srlrvar.data = torch.from_numpy(updated).cuda()

    optimizer.zero_grad()
    srfakep = netP(srlrvar)
    zerovar = Variable(torch.zeros(srfakep.size()).cuda())
    ploss = mseloss(srfakep,zerovar)
    ploss.backward()

    optimizer.step()

    if run%500==0:
        lrimg = srlrvar.data[0,0,:,:].clone().clamp(-1, 1).cpu()
        lrimg = (lrimg + 1) / 2.0 * 255.0
        lrimg = lrimg.numpy().astype("uint8")
        lrimg = Image.fromarray(lrimg, 'L')
        lrimg.show()
        print(ploss)

Variable containing:
 0.1373
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-03 *
  1.3234
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  8.6293
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  6.7465
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  5.6574
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  4.8888
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  4.3196
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.8816
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.5356
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.2533
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  3.0046
[torch.cuda.FloatTensor of size 1 (GPU 1)]

Variable containing:
1.00000e-04 *
  2.7827
[to

In [ ]:
lrimg = Image.open("/home/wcd/Desktop/to_fast_neural_style/images/banckmark/vae/randomLR/comic.bmp").convert('L')
lrimg.show()

In [2]:

from torchvision import transforms
def img2Variable(IMG):
    IMG = transform(IMG)
    IMG = IMG.unsqueeze(0)
    IMG = Variable(IMG, volatile=True)
    return IMG

def imgdir2tensor(img_dir):#file_name=>1x1xhxw
    img = Image.open(img_dir).convert('L')
    return transform(img).unsqueeze(0)

transform = transforms.Compose([
    transforms.ToTensor(),
])

def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])

    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err


def psnr(im1, im2):
    MSE = mse(im1, im2)
    if MSE < 0.00000001:
        psnr = 10000
    else:
        psnr = 20 * np.log10(255) - 10 * np.log10(MSE)
    return psnr


def save_image(filename, data, mode): #from cxhxw tensor
    if mode is 'YCbCr':
        img = data.clone().numpy()
        img = img.transpose(1, 2, 0).astype("uint8")
        Y = Image.fromarray(img[:, :, 0], 'L')
        Cb = Image.fromarray(img[:, :, 1], 'L')
        Cr = Image.fromarray(img[:, :, 2], 'L')
        img = Image.merge('YCbCr', [Y, Cb, Cr]).convert('RGB')
    elif mode is 'RGB':
        img = data.clone().clamp(0, 255).numpy()
        img = img.transpose(1, 2, 0).astype("uint8")
        img = Image.fromarray(img, 'RGB')
    elif mode is 'L':
        img = data.squeeze(0).clone().clamp(0, 255).numpy().astype("uint8")
        img = Image.fromarray(img, 'L')
    img.save(filename)
    return img

In [14]:
np.zeros((4,10,10)).full(2)

AttributeError: 'numpy.ndarray' object has no attribute 'full'